In [240]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [241]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import pycountry



In [3]:
os.getcwd()
#os.chdir("/Users/veesheen/Desktop")

'/Users/veesheenyuen/code/veeyuen/SAA'

In [132]:
country=pycountry.countries.get(name="Lao People's Democratic Republic")

In [133]:
country.alpha_3

'LAO'

# Scraping Wikipedia SEA Games Athletics

In [51]:
# 2009 SEA Games Athletics- Extract using read_html

html = pd.read_html("Athletics at the 2009 SEA Games – Results - Wikipedia.html")

html

[   Rank  Lane              Runner  NOC   Time Notes
 0     1     3  Suryo Agung Wibowo  NaN  10.69     Q
 1     2     6     Wattana Deewong  NaN  10.81     Q
 2     3     5      Amirudin Jamal  NaN  10.83     Q
 3     4     2      Sar Chupveasna  NaN  11.20   NaN
 4     5     4            Kilakone  NaN  11.24   NaN,
    Rank  Lane           Runner  NOC   Time Notes
 0     1     4   Wachara Sondee  NaN  10.61     Q
 1     2     5  Mohd Noor Imran  NaN  10.65     Q
 2     3     2    Poh Seng Song  NaN  10.73     Q
 3     4     6           Fadlin  NaN  10.75     Q
 4     5     3   Bui Duy Thuong  NaN  10.90     Q
 5     6     7      Chaleunsouk  NaN  11.18   NaN,
    Rank  Lane              Runner  NOC   Time Note
 0   NaN     4  Suryo Agung Wibowo  NaN  10.17   GR
 1   NaN     5      Wachara Sondee  NaN  10.30  NaN
 2   NaN     2              Fadlin  NaN  10.61  NaN
 3   4.0     8      Amirudin Jamal  NaN  10.62  NaN
 4   5.0     7       Poh Seng Song  NaN  10.70  NaN
 5   6.0     6    

In [52]:
html_tables = pd.read_html("Athletics at the 2009 SEA Games – Results - Wikipedia.html", match="Rank")

html_tables

[   Rank  Lane              Runner  NOC   Time Notes
 0     1     3  Suryo Agung Wibowo  NaN  10.69     Q
 1     2     6     Wattana Deewong  NaN  10.81     Q
 2     3     5      Amirudin Jamal  NaN  10.83     Q
 3     4     2      Sar Chupveasna  NaN  11.20   NaN
 4     5     4            Kilakone  NaN  11.24   NaN,
    Rank  Lane           Runner  NOC   Time Notes
 0     1     4   Wachara Sondee  NaN  10.61     Q
 1     2     5  Mohd Noor Imran  NaN  10.65     Q
 2     3     2    Poh Seng Song  NaN  10.73     Q
 3     4     6           Fadlin  NaN  10.75     Q
 4     5     3   Bui Duy Thuong  NaN  10.90     Q
 5     6     7      Chaleunsouk  NaN  11.18   NaN,
    Rank  Lane              Runner  NOC   Time Note
 0   NaN     4  Suryo Agung Wibowo  NaN  10.17   GR
 1   NaN     5      Wachara Sondee  NaN  10.30  NaN
 2   NaN     2              Fadlin  NaN  10.61  NaN
 3   4.0     8      Amirudin Jamal  NaN  10.62  NaN
 4   5.0     7       Poh Seng Song  NaN  10.70  NaN
 5   6.0     6    

In [71]:
html_tables[20]['Rank']

0     NaN
1     NaN
2     NaN
3     4.0
4     5.0
5     6.0
6     7.0
7     8.0
8     9.0
9    10.0
Name: Rank, dtype: float64

In [55]:
# Test auto extraction of results for 2009 SEA Games
# Copied over from Asian games distance events

root = 'https://en.wikipedia.org'
URL = f'{root}/wiki/Athletics_at_the_SEA_Games'
# List of events for timed events only
page=requests.get('https://en.wikipedia.org/wiki/Athletics_at_the_2009_SEA_Games_%E2%80%93_Results')
soup=BeautifulSoup(page.content, 'html.parser')

#print(soup)



#LJump_html = pd.read_html("Athletics at the 2018 Asian Games – Men's long jump - Wikipedia.html")

column_names=["Rank", "Name", "Country", "Result", "Qualification", "Round", "Wind", "Heat", "Gender", "Event", "Year", "Competition"]
    
df_master = pd.DataFrame(columns=column_names) # initialize an empty master df

df=pd.DataFrame()

    
URL = f'https://en.wikipedia.org/wiki/Athletics_at_the_2009_SEA_Games_%E2%80%93_Results'    

html = pd.read_html(URL)
        
html_tables = pd.read_html(URL, match="Rank")
    
semi_df=None    
    

# Extract event information

year_pattern='\d\d\d\d'
competition_pattern='SEA Games'
gender_men_pattern='Men'
gender_women_pattern='Women'
event_pattern='\'s\s'

semi_pattern='Semifinals'
final_pattern='Final'
table_pattern='Rank'

    
    
event_list=html[0].columns[0]

print('event_list', event_list)

event=re.split(event_pattern,event_list)[1]

year=re.search(year_pattern, event_list)

event_year=year.group()
    
if competition_pattern in html[0].columns[0]:
    competition='SEA Games'
    
if gender_men_pattern in html[0].columns[0]:
    gender='Male'
else:
    gender='Female'
        
        
num_events=len(html[3])
    
        
for i in range(num_events):    # loop through the one or two tables of results
        
    
    if semi_pattern in html[3]['Event'][i]:
            
        round='Semifinals'
                        
    
    print(round, gender, event, gender, year, competition)
    print(html_tables[i])
    
    if html_tables[i].columns.nlevels>1:  # if more than one column heading drop to one
                
        html_tables[i].columns = html_tables[i].columns.droplevel() 
                
        semi_df = html_tables[i].loc[:, ['Rank', 'Athlete', 'Result', 'Notes']] # Drop useless columns

            
        semi_df['Round']=round  
#            semi_df.append(wind1)  
#            semi_df.append(heat1)
        semi_df['Gender']=gender
        semi_df['Event']=event
        semi_df['Year']=event_year
        semi_df['Competition']=competition
                
#            df_temp.loc[len(df_temp)] = Jump_df
        
#            final=df_temp
                        
            
    if final_pattern in html[3]['Event'][i]:
            
        round='Finals'
            
                        
    if html_tables[i].columns.nlevels>1:  # if more than one column heading drop to one
                
        html_tables[i].columns = html_tables[i].columns.droplevel() 
                
        final_df = html_tables[i].loc[:, ['Rank', 'Athlete', 'Result', 'Notes']] # Drop useless columns
            
            
        final_df['Round']=round  
#            semi_df.append(wind1)  
#            semi_df.append(heat1)
        final_df['Gender']=gender
        final_df['Event']=event
        final_df['Year']=event_year
        final_df['Competition']=competition
            
            
        # Combine the semis and final df    
        
if semi_df is not None:
        
    temp=pd.concat(semi_df, final_df)
        
    df=pd.concat([df, SP_temp])
        
else:
        
    df=pd.concat([df, final_df])
        
        

            
            

        
    





event_list Rank


IndexError: list index out of range

In [56]:
html[1]

,Rank,Lane,Runner,NOC,Time,Notes
0,1,4,Wachara Sondee,NaN,10.61,Q
1,2,5,Mohd Noor Imran,NaN,10.65,Q
2,3,2,Poh Seng Song,NaN,10.73,Q
3,4,6,Fadlin,NaN,10.75,Q
4,5,3,Bui Duy Thuong,NaN,10.90,Q
5,6,7,Chaleunsouk,NaN,11.18,NaN


In [44]:
html_tables[0]

,Rank,Lane,Runner,NOC,Time,Notes
0,1,3,Suryo Agung Wibowo,NaN,10.69,Q
1,2,6,Wattana Deewong,NaN,10.81,Q
2,3,5,Amirudin Jamal,NaN,10.83,Q
3,4,2,Sar Chupveasna,NaN,11.20,NaN
4,5,4,Kilakone,NaN,11.24,NaN


In [49]:
df

,gender,event,heat,results,country
0,Men,100 metres[edit],December 13 - Round One Heats\nDecember 13 - F...,"[1, 3, Suryo Agung Wibowo, , 10.69, Q\n, 2, 6,...","[Suryo Agung Wibowo, Indonesia, Wattana Deewon..."
1,Men,100 metres[edit],Heat 2,"[1, 4, Wachara Sondee, , 10.61, Q\n, 2, 5, Moh...","[Wachara Sondee, Thailand, Mohd Noor Imran (pa..."
2,Men,100 metres[edit],FINAL,"[, 4, Suryo Agung Wibowo, , 10.17, GR\n, , 5, ...","[Suryo Agung Wibowo, Indonesia, Wachara Sondee..."
3,Men,200 metres[edit],December 15 - Round One Heats\nDecember 16 - F...,"[1, 4, Sittichai Suwonprateep, , 21.49, Q\n, 2...","[Sittichai Suwonprateep, Thailand, Sar Chupvea..."
4,Men,200 metres[edit],Heat 2,"[1, 6, Suryo Agung Wibowo, , 21.29, Q\n, 2, 4,...","[Suryo Agung Wibowo, Indonesia, Suppachai Chim..."
...,...,...,...,...,...
98,Women,Heptathlon[edit],200 metres,"[1, Amornrat Winatho (THA), 24.86, 900, \n, 2...","[Amornrat Winatho, Narcisa Atienza (page does ..."
99,Women,Heptathlon[edit],Long jump,"[1, Amornrat Winatho (THA), 5.84, 801, \n, 2,...","[Amornrat Winatho, Duong Thi Viet Anh, Nguyen ..."
100,Women,Heptathlon[edit],Javelin throw,"[1, Narcisa Atienza (PHI), 45.52, 774, \n, 2,...","[Narcisa Atienza (page does not exist), Amornr..."
101,Women,Heptathlon[edit],800 metres,"[1, Amornrat Winatho (THA), 2:23.40, 778, \n,...","[Amornrat Winatho, Nguyen Thi Thu Cuc (page do..."


# Extract 2009 SEAG Results using BS4

In [242]:
from bs4 import BeautifulSoup
import requests

In [243]:
URL ='https://en.wikipedia.org/wiki/Athletics_at_the_2009_SEA_Games_%E2%80%93_Results'
page=requests.get(URL)

In [215]:
page.content

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-appearance-disabled vector-feature-appearance-pinned-clientpref-0 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Athletics at the 2023 SEA Games \xe2\x80\x93 Men\'s Results - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vect

In [244]:
soup=BeautifulSoup(page.content, 'html.parser')


In [25]:
# Creating list with all tables
tables = soup.find_all('table')

#  Looking for the table with the classes 'wikitable' and 'sortable'
table = soup.find('table', class_='wikitable')

In [26]:
for a in soup.find_all('table'):
    print(a.text)




Rank

Lane

Runner

NOC

Time

Notes


1
3
Suryo Agung Wibowo

10.69
Q


2
6
Wattana Deewong

10.81
Q


3
5
Amirudin Jamal

10.83
Q


4
2
Sar Chupveasna

11.20



5
4
Kilakone

11.24




Rank

Lane

Runner

NOC

Time

Notes


1
4
Wachara Sondee

10.61
Q


2
5
Mohd Noor Imran

10.65
Q


3
2
Poh Seng Song

10.73
Q


4
6
Fadlin

10.75
Q


5
3
Bui Duy Thuong

10.90
Q


6
7
Chaleunsouk

11.18




Rank

Lane

Runner

NOC

Time

Note



4
Suryo Agung Wibowo

10.17
GR



5
Wachara Sondee

10.30




2
Fadlin

10.61



4
8
Amirudin Jamal

10.62



5
7
Poh Seng Song

10.70



6
6
Mohd Noor Imran

10.71



7
3
Wattana Deewong

10.73



8
1
Bui Duy Thuong

10.89




Rank

Lane

Runner

NOC

Time

Notes


1
4
Sittichai Suwonprateep

21.49
Q


2
2
Sar Chupveasna

22.94
Q


3
3
Fadlin

22.97
Q


4
4
Souksavanh Tonsaktheva

23.48
Q



Rank

Lane

Runner

NOC

Time

Notes


1
6
Suryo Agung Wibowo

21.29
Q


2
4
Suppachai Chimdee

21.51
Q


3
2
Mohd Noor Imran

22.49
Q


4
3
Bui Duy Thuong

22.76
Q


5

In [29]:
country = soup.find('span', class_='mw-image-border')

print(country)

<span class="mw-image-border" typeof="mw:File"><a href="/wiki/Indonesia" title="Indonesia"><img alt="Indonesia" class="mw-file-element" data-file-height="600" data-file-width="900" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Flag_of_Indonesia.svg/23px-Flag_of_Indonesia.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Flag_of_Indonesia.svg/35px-Flag_of_Indonesia.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Flag_of_Indonesia.svg/45px-Flag_of_Indonesia.svg.png 2x" width="23"/></a></span>


In [30]:
# Extract nation from table
# For those SEAG events where NOC is represented by a flag

for table_tag in soup.find_all('table'):
    for each_row in table_tag.find_all('td'):
        links = each_row.find('a', href=True)
        if links: # check before you access
            title = links.get('title')
            print(title)

Suryo Agung Wibowo
Indonesia
Wattana Deewong (page does not exist)
Thailand
Amirudin Jamal (page does not exist)
Singapore
Sar Chupveasna (page does not exist)
Cambodia
Kilakone (page does not exist)
Laos
Wachara Sondee
Thailand
Mohd Noor Imran (page does not exist)
Malaysia
Poh Seng Song
Singapore
Fadlin
Indonesia
Bui Duy Thuong
Vietnam
Chaleunsouk (page does not exist)
Laos
Suryo Agung Wibowo
Indonesia
Wachara Sondee
Thailand
Fadlin
Indonesia
Amirudin Jamal (page does not exist)
Singapore
Poh Seng Song
Singapore
Mohd Noor Imran (page does not exist)
Malaysia
Wattana Deewong (page does not exist)
Thailand
Bui Duy Thuong
Vietnam
Sittichai Suwonprateep
Thailand
Sar Chupveasna (page does not exist)
Cambodia
Fadlin
Indonesia
Souksavanh Tonsaktheva (page does not exist)
Laos
Suryo Agung Wibowo
Indonesia
Suppachai Chimdee
Thailand
Mohd Noor Imran (page does not exist)
Malaysia
Bui Duy Thuong
Vietnam
Alousana Souvannalat (page does not exist)
Laos
Suryo Agung Wibowo
Indonesia
Sittichai Suwon

In [31]:
table = soup.findChildren('table')[1]

rows = table.findChildren('td')


print(rows)

[<td>1</td>, <td>4</td>, <td><a href="/wiki/Wachara_Sondee" title="Wachara Sondee">Wachara Sondee</a></td>, <td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/wiki/Thailand" title="Thailand"><img alt="Thailand" class="mw-file-element" data-file-height="600" data-file-width="900" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Flag_of_Thailand.svg/23px-Flag_of_Thailand.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Flag_of_Thailand.svg/35px-Flag_of_Thailand.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Flag_of_Thailand.svg/45px-Flag_of_Thailand.svg.png 2x" width="23"/></a></span></span></td>, <td>10.61</td>, <td>Q
</td>, <td>2</td>, <td>5</td>, <td><a class="new" href="/w/index.php?title=Mohd_Noor_Imran&amp;action=edit&amp;redlink=1" title="Mohd Noor Imran (page does not exist)">Mohd Noor Imran</a></td>, <td><span class="flagicon"><span class="mw-image-border" typeof="mw:File

In [32]:
# Loop through all the cells in the table

for row in table.tbody.find_all('tr'):
        # Find all data for each column
        columns = row.find_all('td')
        for td in columns:
            if td.a:
                print(td.a.get('href'))

/wiki/Wachara_Sondee
/wiki/Thailand
/w/index.php?title=Mohd_Noor_Imran&action=edit&redlink=1
/wiki/Malaysia
/wiki/Poh_Seng_Song
/wiki/Singapore
/wiki/Fadlin
/wiki/Indonesia
/wiki/Bui_Duy_Thuong
/wiki/Vietnam
/w/index.php?title=Chaleunsouk&action=edit&redlink=1
/wiki/Laos


In [33]:
# Traversing the hierarchy of the SEAG HTML
# .children, .next_sibling

for object in soup.find_all('h2'):
    for item in object.find_all('span', class_='mw-headline'):
        print(item.text)




Men
Women
References


In [34]:
print(soup.headline)

None


In [245]:
# Latest version for 2009 format htmls
# extracts country when NOC column only shows flag in SEA Games results
# Combines tables from read_html with BS4 for extraction of heat, event and gender details

html_tables = pd.read_html("https://en.wikipedia.org/wiki/Athletics_at_the_2009_SEA_Games_%E2%80%93_Results", match="Rank")

df=pd.DataFrame(columns = ['gender', 'event', 'heat', 'results', 'country'])

list=[]
heat=None
event=None
results=None
country=None
what=None
result_list=[]
countries_list=[]
master_list=[]
heat_list=[]
list=[]
count=0
rank_list=[]

keys=[]
NOC=[]
Nation=[]

for i in soup.find_all('h1'):
    for title in i.find_all('span', class_='mw-page-title-main'): # extract competition
        if title.text:
            print('competition', title.text)

res=re.findall('\d\d\d\d', title.text)
year=res[0]

master_df=pd.DataFrame(columns = ['GENDER', 'EVENT', 'STAGE', 'HEAT', 'RESULT', 'RANK', 'REMARKS', 'COMPETITION', 'NAME', 'DATE'])


for i in soup.find_all('h2'):
    for gender in i.find_all('span', class_='mw-headline'): # gender of event
        if gender.text:
            gend=gender.text
            
        for sib in i.next_siblings:   # h4(events are segregated at h4 level)>ul/li>table>td
            
            # Iterate results table and fill in missing info
            
            if sib.name=='h2':    # event
                event=sib.text
           
        
            
            
            enum = enumerate(html_tables[count]['Rank'])
            
            for ele in enum:
                rank_list.append(ele[0]+1)
            
            # Replace rank column with new rankings column
                        
            html_tables[count].drop('Rank', axis=1, inplace=True)
            
            html_tables[count]['Rank']=pd.Series(rank_list)
            
            
            if sib.name=='h4':    # event
                event=sib.text
                
            elif sib.name == 'ul': # heats and round info
                extract=sib.text
                
                if 'Heat 1' in extract:
                    heat='Heat 1'
                    stage='Heats'
                elif 'Heat 2' in extract:
                    heat='Heat 2'
                    stage='Heats'
                elif 'FINAL' in extract:
                    heat=''
                    stage='Final'
                    
            elif sib.name == 'table':   # results in tables
#                list.append(table.text)
                countries_list=[]
                result_list=[]
        
                for each_result in sib.find_all('td'):  # extract list of countries for each table
                    results=each_result.text  # results
                    result_list.append(results)
                    countries = each_result.find('a', href=True)  # look for a-tag containing country 
                    
                    if countries: # check for non-null
                        
                        country = countries.get('title')
                        countries_list.append(country)
                    
                # Unpack 'countries_list', map to alpha3 code and create dictionary
                    
                
  #              if 'relay' in event:
                if 'Nation' in html_tables[count].columns.values.tolist(): # if relay event
                                        
                    for i in range(0, len(countries_list)-1, 2):
                        
             #           print('relay i', countries_list[i])
            
                        country_name=countries_list[i].strip()     
                
                        if country_name=='Laos':
                            country_name="Lao People's Democratic Republic"
                        
                        if country_name=='Vietnam':
                            country_name="Viet Nam"
                    
                        if country_name=='East Timor':
                            country_name="Timor-Leste"
                    
                    
                        
                        ccode=pycountry.countries.get(name=country_name)
                        country_code=ccode.alpha_3
                                        
                        
                        Nation.append(country_code)  # country column comes before name column for relay events
                        
                        
                
                elif 'NOC' in html_tables[count].columns.values.tolist():          
                
                    for i in range(0, len(countries_list)-1, 2):
                        
                    
                        keys.append(countries_list[i].replace(' (page does not exist)', ''))  # remove unwanted string from name
                    
                        country_name=countries_list[i+1].strip()                    
                    
                    
                        if country_name=='Laos':
                            country_name="Lao People's Democratic Republic"
                        
                        if country_name=='Vietnam':
                            country_name="Viet Nam"
                    
                        if country_name=='East Timor':
                            country_name="Timor-Leste"
                    
                    
                    
                        ccode=pycountry.countries.get(name=country_name)
                        country_code=ccode.alpha_3
                    
                        NOC.append(country_code)
                        
                elif 'Nationality' in html_tables[count].columns.values.tolist():
                    
                    continue
                        
                        
                
                # Add the country column
                
                print('Nation', Nation)
                
                if 'Nation' in html_tables[count].columns.values.tolist() or 'NOC' in html_tables[count].columns.values.tolist(): # if Nation or NOC in columns
                
                    if len(Nation)!=0:  # Replace Nation column 
                    
                        html_tables[count].drop('Nation', axis=1, inplace=True)
            
                        html_tables[count]['Nation']=pd.Series(Nation)
                    else:
                    
                        html_tables[count].drop('NOC', axis=1, inplace=True)
            
                        html_tables[count]['NOC']=pd.Series(NOC)
                
                elif 'Nationality' in html_tables[count].columns.values.tolist(): # skip for throw events
                    
                    continue
                        
                
                dict_name=dict(zip(keys, NOC))
                
                
                # Delete redundant lane column
                
                if 'Lane' in html_tables[count].columns.values.tolist():
                    
                    html_tables[count].drop('Lane', axis=1, inplace=True)

                                    
            
                if gend and event and heat is not None:
                                
#                    list.append(gend)
#                    list.append(event)
#                    list.append(heat)
#                    list.append(result_list)
#                    list.append(countries_list)

                    print('heat', heat)
    
                    html_tables[count]['GENDER']=gend
                    html_tables[count]['EVENT']=event
                    html_tables[count]['HEAT']=heat
                    html_tables[count]['COMPETITION']='SEA Games'
                    html_tables[count]['DATE']=year
                    html_tables[count]['STAGE']=stage
                    
                    
                    html_tables[count].rename(columns = {'NOC':'COUNTRY'}, inplace = True)
                    html_tables[count].rename(columns = {'Nation':'COUNTRY'}, inplace = True)
                    html_tables[count].rename(columns = {'Time':'RESULT'}, inplace = True)
                    html_tables[count].rename(columns = {'Note':'REMARKS'}, inplace = True)
                    html_tables[count].rename(columns = {'Runner':'NAME'}, inplace = True)
                    html_tables[count].rename(columns = {'Rank':'RANK'}, inplace = True)
                    html_tables[count].rename(columns = {'Athlete':'NAME'}, inplace = True)
                    html_tables[count].rename(columns = {'Jumper':'NAME'}, inplace = True)
                    html_tables[count].rename(columns = {'Competitors':'NAME'}, inplace = True)
                    html_tables[count].rename(columns = {'Time':'RESULT'}, inplace = True)
                    html_tables[count].rename(columns = {'Notes':'REMARKS'}, inplace = True)
                    html_tables[count].rename(columns = {'Result':'RESULT'}, inplace = True)
                    html_tables[count].rename(columns = {'Nationality':'COUNTRY'}, inplace = True)

#                    master_df= pd.concat([temp_df, master_df])



                            
                    print(html_tables[count])

                   # TODO: stitch all the results tables together into a consolidated df 
                            
#                    df.loc[len(df)] = list
                    
                    list=[]
                
                count+=1
            rank_list=[]
            NOC=[]
            keys=[]
            Nation=[]

                
                


competition Athletics at the 2009 SEA Games – Results
Nation []
heat Heat 1
                 NAME  RESULT REMARKS  RANK COUNTRY GENDER             EVENT  \
0  Suryo Agung Wibowo   10.69       Q     1     IDN    Men  100 metres[edit]   
1     Wattana Deewong   10.81       Q     2     THA    Men  100 metres[edit]   
2      Amirudin Jamal   10.83       Q     3     SGP    Men  100 metres[edit]   
3      Sar Chupveasna   11.20     NaN     4     KHM    Men  100 metres[edit]   
4            Kilakone   11.24     NaN     5     LAO    Men  100 metres[edit]   

     HEAT COMPETITION  DATE  STAGE  
0  Heat 1   SEA Games  2009  Heats  
1  Heat 1   SEA Games  2009  Heats  
2  Heat 1   SEA Games  2009  Heats  
3  Heat 1   SEA Games  2009  Heats  
4  Heat 1   SEA Games  2009  Heats  
Nation []
heat Heat 2
              NAME  RESULT REMARKS  RANK COUNTRY GENDER             EVENT  \
0   Wachara Sondee   10.61       Q     1     THA    Men  100 metres[edit]   
1  Mohd Noor Imran   10.65       Q     2     

In [297]:
# Latest version for 2023 format htmls
# extracts country when NOC column only shows flag in SEA Games results
# Combines tables from read_html with BS4 for extraction of heat, event and gender details

URL ='https://en.wikipedia.org/wiki/Athletics_at_the_2023_SEA_Games_%E2%80%93_Men%27s_Results'
page=requests.get(URL)
soup=BeautifulSoup(page.content, 'html.parser')


html_tables = pd.read_html("https://en.wikipedia.org/wiki/Athletics_at_the_2023_SEA_Games_%E2%80%93_Men%27s_Results", match="Rank")

df=pd.DataFrame(columns = ['gender', 'event', 'heat', 'results', 'country'])

list=[]
heat=None
event=None
results=None
country=None
what=None
result_list=[]
countries_list=[]
master_list=[]
heat_list=[]
list=[]
count=0
rank_list=[]

keys=[]
NOC=[]
Nation=[]

for i in soup.find_all('h1'):
    for title in i.find_all('span', class_='mw-page-title-main'): # extract gender and year
        if title.text:
            print('competition', title.text)
            search=re.findall('Men|Women', title.text)
            gender=search[0]
            


#res=re.findall('\d\d\d\d', title.text)
#print('year', res)

master_df=pd.DataFrame(columns = ['GENDER', 'EVENT', 'STAGE', 'HEAT', 'RESULT', 'RANK', 'REMARKS', 'COMPETITION', 'NAME', 'DATE'])

# event headings=h2
#'Results' line=h3
# Heat is in <dl>




for i in soup.find_all('h2'):  
    for title in i.find_all('span', class_='mw-headline'):
        if title.text:
            print('event', i.text)   
            event=i.text
        
            
    for sib in i.next_siblings:   # h2(event divider)>dl>table
            
        # Iterate results table and fill in missing info
            
            
        enum = enumerate(html_tables[count]['Rank'])
            
        for ele in enum:
            rank_list.append(ele[0]+1)
            
         # Replace rank column with new rankings column
                        
        html_tables[count].drop('Rank', axis=1, inplace=True)
            
        html_tables[count]['Rank']=pd.Series(rank_list)
        
            
            
        if sib.name=='dl':    # heat
            heat=sib.text 
            print('heat', heat)
                
            if 'Heat 1' in extract:
                heat='Heat 1'
                stage='Heats'
            elif 'Heat 2' in extract:
                heat='Heat 2'
                stage='Heats'
            elif 'FINAL' in extract:
                heat=''
                stage='Final'
            count+=1
                
                    
        elif sib.name == 'table':   # results in tables
     #       print(html_tables[count])
            pass

        
        if sib.name=='h2':    
            next=sib.text 
   #         print('NEXT', heat)

                            
                
            if sib.name=='h2':    # heat
                h2=sib.text 
                print('h2', h2)
                event=h2
    
            
        if gender and event and heat is not None:
                                
#                    list.append(gend)
#                    list.append(event)
#                    list.append(heat)
#                    list.append(result_list)
#                    list.append(countries_list)

   #             print('heat', heat)
    
            html_tables[count]['GENDER']=gender
            html_tables[count]['EVENT']=event
            html_tables[count]['HEAT']=heat
            html_tables[count]['COMPETITION']='SEA Games'
            html_tables[count]['DATE']=year
            html_tables[count]['STAGE']=stage
            
            print(html_tables[count])
            
                    
                    
   #                 html_tables[count].rename(columns = {'NOC':'COUNTRY'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Nation':'COUNTRY'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Time':'RESULT'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Note':'REMARKS'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Runner':'NAME'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Rank':'RANK'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Athlete':'NAME'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Jumper':'NAME'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Competitors':'NAME'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Time':'RESULT'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Notes':'REMARKS'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Result':'RESULT'}, inplace = True)
   #                 html_tables[count].rename(columns = {'Nationality':'COUNTRY'}, inplace = True)

#                    master_df= pd.concat([temp_df, master_df])



                            
                   # TODO: stitch all the results tables together into a consolidated df 
                            
#                    df.loc[len(df)] = list
                    
            list=[]
                
            rank_list=[]
            NOC=[]
            keys=[]
            Nation=[]
            


                
                


competition Athletics at the 2023 SEA Games – Men's Results
event 100 metre[edit]
heat Heat 1
   Rank                     Athlete   Time Notes GENDER            EVENT  \
0     1  Khairul Hafiz Jantan (MAS)  10.37     Q    Men  100 metre[edit]   
1     2       Ngần Ngọc Nghĩa (VIE)  10.47     Q    Men  100 metre[edit]   
2     3      Marc Brian Louis (SGP)  10.52     q    Men  100 metre[edit]   
3     4   Lalu Muhammad Zohri (INA)  10.56     q    Men  100 metre[edit]   
4     5       Anfernee Lopena (PHI)  10.86   NaN    Men  100 metre[edit]   
5     6      Sorsy Phomphakdi (LAO)  10.91   NaN    Men  100 metre[edit]   

     HEAT COMPETITION  DATE  STAGE  
0  Heat 1   SEA Games  2009  Final  
1  Heat 1   SEA Games  2009  Final  
2  Heat 1   SEA Games  2009  Final  
3  Heat 1   SEA Games  2009  Final  
4  Heat 1   SEA Games  2009  Final  
5  Heat 1   SEA Games  2009  Final  
                      Athlete   Time Notes GENDER            EVENT    HEAT  \
0  Khairul Hafiz Jantan (MAS)  10.37

    Rank                              Athlete Round              \
    Rank                              Athlete     1     2     3   
0    NaN                     Janry Ubas (PHI)  7.85     x  7.64   
1    NaN              Nguyễn Tiến Trọng (VIE)  7.54  7.65  7.59   
2    NaN  Sapwaturrahman Sapwaturrahman (INA)  7.53  7.62     x   
3    4.0           Xaidavanh Vongsavanh (LAO)  7.52     x  7.53   
4    5.0                 Pham Van Nghia (VIE)  7.10  7.52  4.48   
5    6.0                 Suwandi Wijaya (INA)  7.39  7.39  7.42   
6    7.0          Shahrizal Nasharuddin (MAS)  6.20  7.21  6.23   
7    8.0      John Marvin Aragon Rafols (PHI)  6.73  7.04  6.39   
8    9.0                 Sunik Muslimin (MAS)  6.94  6.97  6.90   
9   10.0                 Roen Chanrotha (CAM)     x  6.83  6.83   
10  11.0                     Hoeun Chav (CAM)     x  6.47  6.65   
11  12.0                     Hoeun Chav (THA)   DNS   DNS   DNS   

                                                       Mark N

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_25082/2637966903.py:72: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  html_tables[count].drop('Rank', axis=1, inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_25082/2637966903.py:72: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  html_tables[count].drop('Rank', axis=1, inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_25082/2637966903.py:72: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  html_tables[count].drop('Rank', axis=1, inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_25082/2637966903.py:72: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  html_tables[count].drop('Rank', axis=1, inplace=True)
/var

                          Athlete  Round                                 \
                          Athlete      1      2      3                4   
0           Jakkapat Noisri (THA)  17.51      x      x            17.84   
1           Phan Thanh Bình (VIE)  15.65  16.68  17.39            16.37   
2   Muhammad Ziyad Zolkefli (MAS)  17.20  17.12  17.30                x   
3         Jonah Chang Rigan (MAS)  16.27      x  15.82                x   
4   William Edward Morrison (PHI)  15.73      x      x            16.08   
5        Thongchai Silamool (THA)  15.39      x      x            16.06   
6  John Albert Casol Mantua (PHI)      x      x  15.93                x   
7                Sim Samedy (CAM)  14.65      x  13.87            14.65   
8   Pang Chamroeun Vithiear (CAM)  12.28  12.83  13.19  Did not advance   

                                      Mark Notes GENDER             EVENT  \
                 5                6   Mark Notes                            
0                x  

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_25082/2637966903.py:72: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  html_tables[count].drop('Rank', axis=1, inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_25082/2637966903.py:72: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  html_tables[count].drop('Rank', axis=1, inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_25082/2637966903.py:72: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  html_tables[count].drop('Rank', axis=1, inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_25082/2637966903.py:72: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  html_tables[count].drop('Rank', axis=1, inplace=True)
/var

IndexError: list index out of range

In [236]:
html_tables[2]

,NAME,RESULT,REMARKS,RANK,GENDER,EVENT,HEAT,COMPETITION,DATE,STAGE
0,Pen Sokong (CAM),10.80,Q,1,100 metre,100 metre,Heat 1,SEA Games,2023,Final
1,Joshua Hanwei Chua (SGP),10.93,Q,2,100 metre,100 metre,Heat 1,SEA Games,2023,Final
2,Abdul Hakeem Ismail (BRU),11.25,NaN,3,100 metre,100 metre,Heat 1,SEA Games,2023,Final
3,Puripol Boonson (THA),—,DNS,4,100 metre,100 metre,Heat 1,SEA Games,2023,Final
4,Lwin Moe (MYA),—,DNS,5,100 metre,100 metre,Heat 1,SEA Games,2023,Final


### df

In [34]:
countries_list

['Amornrat Winatho',
 'Thailand at the 2009 Southeast Asian Games',
 'Narcisa Matalog Atienza (page does not exist)',
 'Philippines at the 2009 Southeast Asian Games',
 'Nguyen Thi Thu Cuc (page does not exist)',
 'Vietnam at the 2009 Southeast Asian Games',
 'Duong Thi Viet Anh',
 'Vietnam at the 2009 Southeast Asian Games',
 'Philaylack Sackpaseuth',
 'Laos at the 2009 Southeast Asian Games',
 'List of Laotian records in athletics']

In [36]:
keys=[]
countries=[]
for i in range(0, len(countries_list)-1, 2):
    keys.append(countries_list[i])
    countries.append(countries_list[i+1])
    

In [37]:
keys

['Amornrat Winatho',
 'Narcisa Matalog Atienza (page does not exist)',
 'Nguyen Thi Thu Cuc (page does not exist)',
 'Duong Thi Viet Anh',
 'Philaylack Sackpaseuth']

In [38]:
countries

['Thailand at the 2009 Southeast Asian Games',
 'Philippines at the 2009 Southeast Asian Games',
 'Vietnam at the 2009 Southeast Asian Games',
 'Vietnam at the 2009 Southeast Asian Games',
 'Laos at the 2009 Southeast Asian Games']

In [39]:
dict_name=dict(zip(keys, countries))

In [40]:
dict_name

{'Amornrat Winatho': 'Thailand at the 2009 Southeast Asian Games',
 'Narcisa Matalog Atienza (page does not exist)': 'Philippines at the 2009 Southeast Asian Games',
 'Nguyen Thi Thu Cuc (page does not exist)': 'Vietnam at the 2009 Southeast Asian Games',
 'Duong Thi Viet Anh': 'Vietnam at the 2009 Southeast Asian Games',
 'Philaylack Sackpaseuth': 'Laos at the 2009 Southeast Asian Games'}

In [13]:
df

,gender,event,heat,results,country
0,Men,100 metres[edit],December 13 - Round One Heats\nDecember 13 - F...,"[1, 3, Suryo Agung Wibowo, , 10.69, Q\n, 2, 6,...","[Suryo Agung Wibowo, Indonesia, Wattana Deewon..."
1,Men,100 metres[edit],Heat 2,"[1, 4, Wachara Sondee, , 10.61, Q\n, 2, 5, Moh...","[Wachara Sondee, Thailand, Mohd Noor Imran (pa..."
2,Men,100 metres[edit],FINAL,"[, 4, Suryo Agung Wibowo, , 10.17, GR\n, , 5, ...","[Suryo Agung Wibowo, Indonesia, Wachara Sondee..."
3,Men,200 metres[edit],December 15 - Round One Heats\nDecember 16 - F...,"[1, 4, Sittichai Suwonprateep, , 21.49, Q\n, 2...","[Sittichai Suwonprateep, Thailand, Sar Chupvea..."
4,Men,200 metres[edit],Heat 2,"[1, 6, Suryo Agung Wibowo, , 21.29, Q\n, 2, 4,...","[Suryo Agung Wibowo, Indonesia, Suppachai Chim..."
...,...,...,...,...,...
98,Women,Heptathlon[edit],200 metres,"[1, Amornrat Winatho (THA), 24.86, 900, \n, 2...","[Amornrat Winatho, Narcisa Atienza (page does ..."
99,Women,Heptathlon[edit],Long jump,"[1, Amornrat Winatho (THA), 5.84, 801, \n, 2,...","[Amornrat Winatho, Duong Thi Viet Anh, Nguyen ..."
100,Women,Heptathlon[edit],Javelin throw,"[1, Narcisa Atienza (PHI), 45.52, 774, \n, 2,...","[Narcisa Atienza (page does not exist), Amornr..."
101,Women,Heptathlon[edit],800 metres,"[1, Amornrat Winatho (THA), 2:23.40, 778, \n,...","[Amornrat Winatho, Nguyen Thi Thu Cuc (page do..."


In [14]:
object=soup.find_all('h2')

object

[<h2 class="vector-pinnable-header-label">Contents</h2>,
 <h2><span class="mw-headline" id="Men">Men</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=Athletics_at_the_2009_SEA_Games_%E2%80%93_Results&amp;action=edit&amp;section=1" title="Edit section: Men"><span>edit</span></a><span class="mw-editsection-bracket">]</span></span></h2>,
 <h2><span class="mw-headline" id="Women">Women</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=Athletics_at_the_2009_SEA_Games_%E2%80%93_Results&amp;action=edit&amp;section=29" title="Edit section: Women"><span>edit</span></a><span class="mw-editsection-bracket">]</span></span></h2>,
 <h2><span class="mw-headline" id="References">References</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=Athletics_at_the_2009_SEA_Games_%E2%80%93_Results&amp;action=edit&amp;section=56" title="Edi

In [18]:
table = soup.find_all("table", class_="wikitable sortable jquery-tablesorter")

for tds in table:
    td = tds.find_all("td")
#    print(td.get_text(strip=True))
    print(td)

In [19]:
URL ='https://en.wikipedia.org/wiki/Athletics_at_the_2009_SEA_Games_%E2%80%93_Results'
page=requests.get(URL)

soup=BeautifulSoup(page.content, 'html.parser')

for div in soup.find_all('div', attrs={ "class" : "mw-content-ltr mw-parser-output"}):
    h3tag = div.find_all('h3')
    marker = div.find_all('ul')
    h4tag = div.find_all('h4')
    tabletag = div.find_all('table')

    
    
    
for i in h3tag:
    print(i.text)
    for j in h4tag:
        print(j.text)
for l in marker:
    output = l.find('li')
    print(output.text)
    
    
#for table in tabletag:
#    print(table.text)



Track Events[edit]
100 metres[edit]
200 metres[edit]
400 metres[edit]
800 metres[edit]
1500 metres[edit]
5000 metres[edit]
10,000 metres[edit]
110 metre hurdles[edit]
400 metre hurdles[edit]
3000 metre steeplechase[edit]
4×100 metre relay[edit]
4×400 metre relay[edit]
Marathon[edit]
20 km walk[edit]
High jump[edit]
Pole vault[edit]
Long jump[edit]
Triple jump[edit]
Shot put[edit]
Discus throw[edit]
Hammer throw[edit]
Javelin throw[edit]
Decathlon[edit]
100 metres[edit]
200 metres[edit]
400 metres[edit]
800 metres[edit]
1500 metres[edit]
5000 metres[edit]
10,000 metres[edit]
100 metre hurdles[edit]
400 metre hurdles[edit]
4×100 metre relay[edit]
4×400 metre relay[edit]
Marathon[edit]
20 km walk[edit]
High jump[edit]
Pole vault[edit]
Long jump[edit]
Triple jump[edit]
Shot put[edit]
Discus throw[edit]
Hammer throw[edit]
Javelin throw[edit]
Heptathlon[edit]
Race Walk[edit]
100 metres[edit]
200 metres[edit]
400 metres[edit]
800 metres[edit]
1500 metres[edit]
5000 metres[edit]
10,000 metres[

In [20]:
URL ='https://en.wikipedia.org/wiki/Athletics_at_the_2009_SEA_Games_%E2%80%93_Results'
page=requests.get(URL)

pattern = 'h3'


html_parsed=BeautifulSoup(page.content, 'html.parser')
html = BeautifulSoup(str(html_parsed), 'html.parser')

table = html.find('table', class_='wikitable sortable jquery-tablesorter')

tr=soup.findAll('table')[1].findAll('tr')

#print(tr)
for content in tr:
    
    if content.a:
        print(content.a.get('href'))

#for row in table.tbody.find_all('tr'):
    
    # Find all data for each column
#    columns = row.find_all('td')
#    for td in columns:
#        if td.a:
#            print(td.a.get('href'))

#for href in html.find_all('a', href=True):
#    print(href.text)


/wiki/Wachara_Sondee
/w/index.php?title=Mohd_Noor_Imran&action=edit&redlink=1
/wiki/Poh_Seng_Song
/wiki/Fadlin
/wiki/Bui_Duy_Thuong
/w/index.php?title=Chaleunsouk&action=edit&redlink=1


In [16]:
URL ='https://en.wikipedia.org/wiki/Athletics_at_the_2009_SEA_Games_%E2%80%93_Results'
page=requests.get(URL)

pattern = 'h3'


html_parsed=BeautifulSoup(page.content, 'html.parser')
html = BeautifulSoup(str(html_parsed), 'html.parser')

for div in html.find_all('div', attrs={ "class" : "mw-content-ltr mw-parser-output"}):
    h2tag = div.find_all('h2')
    h3tag = div.find_all('h3')
    marker = div.find_all('ul')
    h4tag = div.find_all('h4')
    
    tabletag = div.find_all('table')

    
    
for k in h2tag:
    print(k.text)
for i in h3tag:
    print(i.text)
for j in h4tag:
    print(j.text)
for l in marker:
    output = l.find('li')
    print(output.text)
    


Men[edit]
Women[edit]
References[edit]
Track Events[edit]
Race Walk[edit]
Field Events[edit]
Combined Event[edit]
Track Events[edit]
Race Walk[edit]
Field Event[edit]
Combined Event[edit]
100 metres[edit]
200 metres[edit]
400 metres[edit]
800 metres[edit]
1500 metres[edit]
5000 metres[edit]
10,000 metres[edit]
110 metre hurdles[edit]
400 metre hurdles[edit]
3000 metre steeplechase[edit]
4×100 metre relay[edit]
4×400 metre relay[edit]
Marathon[edit]
20 km walk[edit]
High jump[edit]
Pole vault[edit]
Long jump[edit]
Triple jump[edit]
Shot put[edit]
Discus throw[edit]
Hammer throw[edit]
Javelin throw[edit]
Decathlon[edit]
100 metres[edit]
200 metres[edit]
400 metres[edit]
800 metres[edit]
1500 metres[edit]
5000 metres[edit]
10,000 metres[edit]
100 metre hurdles[edit]
400 metre hurdles[edit]
4×100 metre relay[edit]
4×400 metre relay[edit]
Marathon[edit]
20 km walk[edit]
High jump[edit]
Pole vault[edit]
Long jump[edit]
Triple jump[edit]
Shot put[edit]
Discus throw[edit]
Hammer throw[edit]
Ja

In [21]:
h3tag

[<h3><span class="mw-headline" id="Track_Events">Track Events</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=Athletics_at_the_2009_SEA_Games_%E2%80%93_Results&amp;action=edit&amp;section=2" title="Edit section: Track Events"><span>edit</span></a><span class="mw-editsection-bracket">]</span></span></h3>,
 <h3><span class="mw-headline" id="Race_Walk">Race Walk</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=Athletics_at_the_2009_SEA_Games_%E2%80%93_Results&amp;action=edit&amp;section=16" title="Edit section: Race Walk"><span>edit</span></a><span class="mw-editsection-bracket">]</span></span></h3>,
 <h3><span class="mw-headline" id="Field_Events">Field Events</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=Athletics_at_the_2009_SEA_Games_%E2%80%93_Results&amp;action=edit&amp;section=18" title="Edit section: Fie

In [24]:
print(div.find_all_next("h3"))


[<h3><span class="mw-headline" id="Track_Events">Track Events</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=Athletics_at_the_2009_SEA_Games_%E2%80%93_Results&amp;action=edit&amp;section=2" title="Edit section: Track Events"><span>edit</span></a><span class="mw-editsection-bracket">]</span></span></h3>, <h3><span class="mw-headline" id="Race_Walk">Race Walk</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=Athletics_at_the_2009_SEA_Games_%E2%80%93_Results&amp;action=edit&amp;section=16" title="Edit section: Race Walk"><span>edit</span></a><span class="mw-editsection-bracket">]</span></span></h3>, <h3><span class="mw-headline" id="Field_Events">Field Events</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=Athletics_at_the_2009_SEA_Games_%E2%80%93_Results&amp;action=edit&amp;section=18" title="Edit section: Field

In [25]:
tabletag[4].text

'\n\nRank\n\nLane\n\nRunner\n\nNOC\n\nTime\n\nNotes\n\n\n1\n6\nSuryo Agung Wibowo\n\n21.29\nQ\n\n\n2\n4\nSuppachai Chimdee\n\n21.51\nQ\n\n\n3\n2\nMohd Noor Imran\n\n22.49\nQ\n\n\n4\n3\nBui Duy Thuong\n\n22.76\nQ\n\n\n5\n5\nAlousana Souvannalat\n\n42.00\n\n'

In [26]:

for par in [e for e in html if e.find('h3')]:

    sib = par.find_next_siblings()
    filtered = []
    for e in sib:
        if not e.find('h3'):
            filtered.append(e)
        else:
            break

#    print(f'the paragraph is {par}')
#    print(f'next paragraphs are :{filtered}')
#    print('###')
#    print('         ')

In [27]:
par

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-appearance-disabled vector-feature-appearance-pinned-clientpref-0 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Athletics at the 2009 SEA Games – Results - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vec

In [353]:
html_doc = '''
This I <b>don't</b> want
<h1></h1>
Text <i>here</i> has no tag
<div>This is in a div</div>
<h1></h1>
This I <b>don't</b> want too
'''

soup = BeautifulSoup(html_doc, 'html.parser')

for c in list(soup.contents):  # remove unwanted tag from soup
    if c is soup.h1 or c.find_previous('h1') is soup.h1:
        continue
    c.extract() # pops tag from list

for h1 in soup.select('h1'):  
    h1.extract()

print(soup)


Text <i>here</i> has no tag
<div>This is in a div</div>



In [28]:
# TEST Scrape HTML and output each section as a separate list of list
# Check how HTML is segmented...h2 (for distance events) or h3 (for timed events) tags

import re

URL ='https://en.wikipedia.org/wiki/Athletics_at_the_2023_SEA_Games'
page=requests.get(URL)

pattern = 'h3'

sections=[]

table=[]

html_parsed=BeautifulSoup(page.content, 'html.parser')
html = BeautifulSoup(str(html_parsed), 'html.parser')

for div in html.find_all('div', attrs={ "class" : "mw-content-ltr mw-parser-output"}):
    
    info=html.find('span', attrs={ "class" : "mw-page-title-main"})  # find info for event
    
    sections.append(info.text)
    
    h2s = html.find_all(pattern)

    for h2 in h2s:  # loop each h2 section
        temp_data = [h2.text]
        
        for tag in h2.next_siblings:  # look for siblings after each h3
            
            row=[]
                        
            if tag.name == 'table':
    
                if tag.text is not None:

                    row.append(tag.text.replace('\n', ' '))
                
                    table.append(row)
                
            if tag.name == 'div':  # don't append div class text
                continue    
                            
            if tag.name == 'h2':
                
                break # exit and move onto the next position in the list
        
            else:
                
                
                if tag.text is not None:

                
                   # temp_data.append(tag.text)
                    temp_data.append(tag.text.replace('\n', ' '))

                    
            
        sections.append(temp_data) # each section of html is appended as a list

In [29]:
table[1]

['  Rank  Lane  Runner  NOC  Time  Notes   1 4 Wachara Sondee  10.61 Q   2 5 Mohd Noor Imran  10.65 Q   3 2 Poh Seng Song  10.73 Q   4 6 Fadlin  10.75 Q   5 3 Bui Duy Thuong  10.90 Q   6 7 Chaleunsouk  11.18  ']

In [43]:
table[5]

['  Rank  Lane  Runner  NOC  Time  Note    4 Suryo Agung Wibowo  20.85     3 Sittichai Suwonprateep  21.12     5 Suppachai Chimdee  21.24    4 8 Mohd Noor Imran  21.60    5 7 Fadlin  22.25    6 1 Bui Duy Thuong  22.56    7 6 Sar Chupveasna  22.85    8 2 Souksavanh Tonsaktheva  23.44  ']

In [44]:
info

<span class="mw-page-title-main">Athletics at the 2009 SEA Games – Results</span>

In [45]:
sections[0]

'Athletics at the 2009 SEA Games – Results'

In [46]:
sections[1][20]

'  Rank  Lane  Runner  NOC  Time  Notes   1 4 Sittichai Suwonprateep  21.49 Q   2 2 Sar Chupveasna  22.94 Q   3 3 Fadlin  22.97 Q   4 4 Souksavanh Tonsaktheva  23.48 Q '

In [47]:
sections[2]

['Race Walk[edit]',
 ' ',
 '20 km walk[edit]',
 ' ',
 'December 15 - FINAL FINAL',
 ' ',
 '  Rank  Lane  Runner  NOC  Time  Notes    5 Teoh Boon Lim  01:31:28     1 Indra Abdul Kadir  01:32:09     2 Kristian Lumbang Tobing  01:33:35    4 6 Michael Embuedo  01:41:54    5 7 Veerapun Anunchai  01:46:08    6 3 Bountome Singlokham  02:00:40    7 4 Lo Choon Sieng   DQ  ',
 ' ',
 'Field Events[edit]',
 ' ',
 'High jump[edit]',
 ' ',
 'December 17 - FINAL FINAL',
 ' ',
 '  Rank  Athlete  NOC  Result  Notes    Lee Hup Wei  2.18m     Pramote Pumurai  2.14m     Suchart Singhaklang  2.10m    4 Nguyen Duy Bang  2.10m    5 Navinraj Subramaniam  2.05m    6 Syahrial  2.05m  ',
 ' ',
 'Pole vault[edit]',
 ' ',
 'December 16 - FINAL',
 ' ',
 '  Rank Jumper Result Notes     Kreeta Sintawacheewa  5.21m GR     Sompong Sombankuay  5.00m      Muhd Hafizuddin  4.50m    4  Tun Tun Lin   5  Ying Yot Novom   6  Sean Lim   7  Vu Van Huyen  ',
 ' ',
 'Long jump[edit]',
 ' ',
 'December 15 - FINAL',
 ' ',
 '  Rank 

In [48]:
sections[3]

['Field Events[edit]',
 ' ',
 'High jump[edit]',
 ' ',
 'December 17 - FINAL FINAL',
 ' ',
 '  Rank  Athlete  NOC  Result  Notes    Lee Hup Wei  2.18m     Pramote Pumurai  2.14m     Suchart Singhaklang  2.10m    4 Nguyen Duy Bang  2.10m    5 Navinraj Subramaniam  2.05m    6 Syahrial  2.05m  ',
 ' ',
 'Pole vault[edit]',
 ' ',
 'December 16 - FINAL',
 ' ',
 '  Rank Jumper Result Notes     Kreeta Sintawacheewa  5.21m GR     Sompong Sombankuay  5.00m      Muhd Hafizuddin  4.50m    4  Tun Tun Lin   5  Ying Yot Novom   6  Sean Lim   7  Vu Van Huyen  ',
 ' ',
 'Long jump[edit]',
 ' ',
 'December 15 - FINAL',
 ' ',
 '  Rank Athlete Nationality 1 2 3 4 5 6 Result Notes    Supanara Sukhasvasti  Thailand 7.50 X 7.70 X X 7.83 7.83     Joebert Delicano  Philippines 7.61 7.74 7.61 X 7.72 7.61 7.74     Henry Dagmil  Philippines 6.97 X 7.53 7.32 X 7.72 7.72    4 Nguyen Van Mua  Vietnam 7.42 7.65 X X 7.50 X 7.65    5 Theerayut Philakong  Thailand X X 7.50 X 7.64 X 7.64    6 Matthew Goh  Singapore 7.62

In [49]:
pd.DataFrame(sections[2])

,0
0,Race Walk[edit]
1,
2,20 km walk[edit]
3,
4,December 15 - FINAL FINAL
...,...
101,
102,Standings after Event 1014 December 2009
103,
104,Rank Athlete NOC December 13 December 14...


In [213]:
# HTML parser function

def html_parser(html):

# h2 = 1500 meters    
    
    global output
    
    h3_pattern = 'h3'

    data=[]

    table=[]

    for div in html.find_all('div', attrs={ "class" : "mw-content-ltr mw-parser-output"}):
    
        info=html.find('span', attrs={ "class" : "mw-page-title-main"})  # find info for event
    
        data.append(info.text)
    
        h3s = html.find_all('h3')

        for h3 in h3s:  # loop each h3 section
            temp_data = [h3.text]
        
            for tag in h3.next_siblings:  # look for siblings after each h3
            
                row=[]
                        
                if tag.name == 'table':
    
                    if tag.text is not None:

                        row.append(tag.text.replace('\n', ' '))
                
                        table.append(row)
                
                if tag.name == 'div':  # don't append div class text
                    continue    
                            
                if tag.name == 'h3':
                
                    break # exit and move onto the next position in the list
        
                else:
                
                
                    if tag.text is not None:

                
                   # temp_data.append(tag.text)
                        temp_data.append(tag.text.replace('\n', ' '))

                    
            
            data.append(temp_data) # each section of html is appended as a list
            
            output=data
                        
        return output

In [30]:
output

<li><a href="/wiki/List_of_SEA_Games_medalists_in_athletics" title="List of SEA Games medalists in athletics">List of gold medallists</a></li>

In [43]:
def remove_accents(input_str):
    s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
    s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
    s = ''
    print(input_str.encode('utf-8'))
    for c in input_str:
        if c in s1:
            s += s0[s1.index(c)]
        else:
            s += c
    return s

#def remove_accents(input_str):
#    nfkd_form = unicodedata.normalize('NFKD', input_str)
#    return u"".join(c for c in nfkd_form if not unicodedata.combining(c))


In [210]:
# Test HTML parser function designed for h2 or h3

def html_parser_test(html, pattern):

# h2 = 1500 meters    
    
    global output
    
#    pattern = 'h3' or 'h2'

    data=[]

    table=[]

    for div in html.find_all('div', attrs={ "class" : "mw-content-ltr mw-parser-output"}):
    
        info=html.find('span', attrs={ "class" : "mw-page-title-main"})  # find info for event
    
        data.append(info.text)
    
        pattern_s = html.find_all(pattern)

        for pattern in pattern_s:  # loop through each h2/h3 section
            temp_data = [pattern.text]
        
            for tag in pattern.next_siblings:  # look for siblings after each h2/h3
            
                row=[]
                        
                if tag.name == 'table':
    
                    if tag.text is not None:

                        row.append(tag.text.replace('\n', ' '))
                
                        table.append(row)
                
                if tag.name == 'div':  # don't append div class text
                    continue    
                            
                if tag.name == pattern:
                
                    break # exit and move onto the next position in the list
        
                else:
                
                
                    if tag.text is not None:

                
                   # temp_data.append(tag.text)
                        temp_data.append(tag.text.replace('\n', ' '))

                    
            
            data.append(temp_data) # each section of html is appended as a list
            
            output=data
                        
        return output

In [212]:
def results(data, round, wind1, heat1, gender, event, year, competition):
    
    global final

    temp_names=["Rank", "Name", "Country", "Result", "Qualification", "Round", "Wind", "Heat", "Gender", "Event", "Year", "Competition"]
    
    df_temp = pd.DataFrame(columns=temp_names)


# turn results into a list and strip out column headers
# replace '\n' with ' ' beforehand

    pattern='\s\xa0'
    
    name_pattern='[a-zA-Z\-\. \"]{3,31}'


    delimiter=''
    string=delimiter.join(data)

    res=re.split(pattern, string)

    res.pop(0) # remove column names
        
# Create dataframe


    for count, ele in enumerate(res):
        
        unicode = remove_accents(ele)
                
        lpos = re.search('^[A-Z]', unicode) # UNICODE to strip accented characters. First capital letter in name
        
        
        
        
        print('lpos', lpos)
        pattern = '\xa0'
        
        if 'DNS' in ele or 'DNF' in ele or 'DSQ' in ele or 'DQ' in ele:
            
            rpos = re.search('DNS|DSQ|DNF|DQ', ele)
            
            res = ele[rpos.start():] # the DNF/DSQ/DNF/DQ
    
                 
            name=ele[lpos.start():rpos.start()] # extract competitor name
            
            sp=re.split(pattern, name) # split name from country
            
            count=str(count+1) # increment rank
            
            combined=count + l + ',' + sp[0] +','+ sp[1]+', '+ res +', '+' ' # rejoin all the elements of the list - Rank, Name, Country, Result, Qualification


           
        
        else:
            
            rpos = re.search('\s\d', ele) # First digit in result
            
            l = ele[:lpos.start()] # left string post splicing
        

            
            r = ele[rpos.start():] # right string post slicing
    
                 
            name=ele[lpos.start():rpos.start()] # extract competitor name
            
            sp=re.split(pattern, name) # split name from country
            
        
            list=r.split(' ')  # result is in this list
            
                    
        
            
            count=str(count+1) # increment rank

            combined=count + l + ',' + sp[0] +','+ sp[1]+', '.join(list) # rejoin all the elements of the list - Rank, Name, Country, Result, Qualification




        
      
                
        # Splice string to extract name of competitor
        # Use a different process for relay events due to names being concatenated with spaces
        
            
        
        
        line=combined.split(",")
            
            
            
            
    
    
        K=len(line)-5
        
        new_row = line[: len(line) - K] # drop last few empty entries from list
        
        new_row.append(round)  
        new_row.append(wind1)  
        new_row.append(heat1)
        new_row.append(gender)
        new_row.append(event)
        new_row.append(event_year)
        new_row.append(competition)
        
                                    
        df_temp.loc[len(df_temp)] = new_row
        
        final=df_temp
            
    
    return final
            

        
    

In [45]:
# loop through each element in html section and create dataframe

column_names=["Rank", "Name", "Country", "Result", "Qualification", "Round", "Wind", "Heat", "Gender", "Event", "Year", "Competition"]
    
df_master = pd.DataFrame(columns=column_names) # initialize an empty master df


heat_pattern='Heat(\s\d)'
wind_pattern='(?<=\Wind:\s)(\D|\W)\d{1}.\d{1}'
round_pattern='Round 1' or '1st Round'
semi_pattern='Semifinals'
final_pattern='Final'
table_pattern='Rank'


# Extract event information

year_pattern='\d\d\d\d'
competition_pattern='Asian Games'
gender_men_pattern='Men\'s'
gender_women_pattern='Women\'s'
event_pattern='\'s\s'


event_list=data[0]

event=re.split(event_pattern,event_list)[1]

year=re.search(year_pattern, event_list)

event_year=year.group()

if competition_pattern in data[0]:
    competition='Asian Games'
    
if gender_men_pattern in data[0]:
    gender='Male'
else:
    gender='Female'


# Extract competition results
    
for j in range(1, len(data)): # iterate over each section (header, round1, semis and final)
        
    length=len(data[j])
    

    for i in range(length):  # iterate over each entry in each section of the html
    
        heat=re.search(heat_pattern, data[j][i])
        wind=re.search(wind_pattern, data[j][i])
        rank=re.search(table_pattern, data[j][i])
        
        print(wind)
        
    # extract round info
    
        if round_pattern in data[j][i]:
        
            round='Round 1'
        
        elif semi_pattern in data[j][i]:
        
            round='Semifinals'
        
        elif final_pattern in data[j][i]:
        
            round='Final'
        
    # extract heats data

        if heat:
        
            heat1=heat.group()
        
        
    # extract wind data

        if wind:
        
            wind1=wind.group()  
        
                
        if rank:
                            
#                    pattern='\d\s\xa0'


#                    test=table[0]

#                    delimiter=' '
#                    string=delimiter.join(test)

#                    res=re.split(pattern, string)

                    
            df_new=results(data[j][i], round, wind1, heat1, gender, event, event_year, competition)
        
            df_master = pd.concat([df_master, df_new], axis=0)
 
                             

NameError: name 'data' is not defined

In [189]:
df_master.tail(45)

,Rank,Name,Country,Result,Qualification,Round,Wind,Heat,Gender,Event,Year,Competition


In [208]:
# TEST scrape all timed event links

root = 'https://en.wikipedia.org'
URL = f'{root}/wiki/Athletics_at_the_2023_SEA_Games#References'

page=requests.get(URL)

soup=BeautifulSoup(page.content, 'html.parser')

# List of events for timed events only

time_list=['100_metres','200_metres', '400_metres', '800_metres', '1500_metres', '5000_metres', '10,000_metres', '100_metres_hurdles', 
          '110_metres_hurdles', '400_metres_hurdles', 'steeplechase', 'relay', 'Marathon']

links = [link['href'] for link in soup.find_all('a', href=re.compile("_Women"))]  # compile all the links for women events into a list

column_names=["Rank", "Name", "Country", "Result", "Qualification", "Round", "Wind", "Heat", "Gender", "Event", "Year", "Competition"]
    
df_master = pd.DataFrame(columns=column_names) # initialize an empty master df


# 1500 metres has h2 and not h3 as dividers

for link in links:  # iterate through each link
    
        
    if any(x in link for x in time_list):  # check if the link belongs to a time event
        
        print(link)
                
        result = requests.get(f'{root}/{link}')
        content = result.text
        html = BeautifulSoup(result.content, 'html.parser')
    
        if 'hurdles' or '1500' in link:    # if hurdles or 1500 metres event use 'h2' to splice html
    
            pattern='h2'
        
        else:
            
            pattern='h3'
    
        html_parser_test(html, pattern) # invoke function to parse html into sections
    
        data=output # 'output' is df returned from above function call

        heat_pattern='Heat(\s\d)'
        wind_pattern='(?<=Wind:\s)\S\d{1}.\d{1}'
        round_pattern='Round 1'
        semi_pattern='Semifinals'
        final_pattern='Final'
        table_pattern='Rank'


# Extract event information

        year_pattern='\d\d\d\d'
        competition_pattern='Asian Games'
        gender_men_pattern='Men\'s'
        gender_women_pattern='Women\'s'
        event_pattern='\'s\s'


        event_list=data[0]
        
        print(data[0])

        event=re.split(event_pattern,event_list)[1]

        year=re.search(year_pattern, event_list)

        event_year=year.group()
    
        if competition_pattern in data[0]:
            competition='Asian Games'
    
        if gender_men_pattern in data[0]:
            gender='Male'
        else:
            gender='Female'


# Extract competition results
    
        for j in range(1, len(data)): # iterate over each html section. Skip first entry as that contains header info.
        
            wind1=None
            round=None
        
            length=len(data[j])    

            for i in range(length):  # iterate over each list entry in each section of the html
                
                heat=re.search(heat_pattern, data[j][i])
                wind=re.search(wind_pattern, data[j][i])
                rank=re.search(table_pattern, data[j][i])
                        
    # extract round info
    
                if round_pattern in data[j][i]:
        
                    round='Round 1'
        
                elif semi_pattern in data[j][i]:
        
                    round='Semifinals'
        
                elif final_pattern in data[j][i]:
        
                    round='Final'
        
    # extract heats data

                if heat:
        
                    heat1=heat.group()
            
        
    # extract wind data

                if wind:
                    
                    wind1=wind.group()  
        
    # extract table data
    
                if rank:  # if a table is found
                            

                    if round=='Final':# reset heat to final if at finals
                        heat1='Final'
                        
                    if 'relay' not in link:
                                                
                        df_new=results(data[j][i], round, wind1, heat1, gender, event, event_year, competition)
        
                        df_master = pd.concat([df_master, df_new], axis=0)
            
                    else:
                        
                        print(data[j][i])
                    
                        df_new=results_relay(data[j][i], round, wind1, heat1, gender, event, event_year, competition)

                        df_master = pd.concat([df_master, df_new], axis=0)

                    
                        
            
                             
    
    
    

/wiki/Athletics_at_the_2023_SEA_Games_%E2%80%93_Women%27s_Results#3000_metre_steeplechase


NameError: name 'html_parser_test' is not defined

In [104]:
df_master

,Rank,Name,Country,Result,Qualification,Round,Wind,Heat,Gender,Event,Year,Competition
0,1,Susanthika Jayasinghe,(SRI),11.16,Q,Final,+0.6,Final,Female,100 metres,2002,Asian Games
1,2,Qin Wangping,(CHN),11.54,Q,Final,+0.6,Final,Female,100 metres,2002,Asian Games
2,3,Viktoriya Kovyreva,(KAZ),11.56,Q,Final,+0.6,Final,Female,100 metres,2002,Asian Games
3,4,Saraswati Saha,(IND),11.61,q,Final,+0.6,Final,Female,100 metres,2002,Asian Games
4,5,Orranut Klomdee,(THA),11.64,q,Final,+0.6,Final,Female,100 metres,2002,Asian Games
...,...,...,...,...,...,...,...,...,...,...,...,...
2,3,"[Nguyễn Thị Oanh, Nguyễn Thị Hằng, Hoàng Thị N...",(VIE),3:33.23,,None,None,Final,Female,4 × 400 metres relay,2018,Asian Games
3,4,"[Liang Nuo, Cheng Chong, Pan Gaoqin, Huang Gui...",(CHN),3:33.72,,None,None,Final,Female,4 × 400 metres relay,2018,Asian Games
4,5,"[Ayaka Kawata, Yume Kitamura, Eri Utsunomiya, ...",(JPN),3:34.14,,None,None,Final,Female,4 × 400 metres relay,2018,Asian Games
5,6,"[Lyubov Ushakova, Margarita Mukasheva, Svetlan...",(KAZ),3:36.73,,None,None,Final,Female,4 × 400 metres relay,2018,Asian Games


In [105]:
df_master[df_master["Event"]=='Marathon']

,Rank,Name,Country,Result,Qualification,Round,Wind,Heat,Gender,Event,Year,Competition


# Test special character removal

In [51]:
test="Đỗ Thị Bông (VIE)  2:08.64 Q   3 "

In [52]:
unicode = remove_accents(test3)

unicode

NameError: name 'test3' is not defined

In [53]:
lpos = re.search('^[A-Z]', unicode)

NameError: name 'unicode' is not defined

In [54]:
lpos2 = re.search('\s\s[A-Z]', unicode)

NameError: name 'unicode' is not defined

In [55]:
lpos2

NameError: name 'lpos2' is not defined

In [202]:
def remove_accents(input_str):
    s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
    s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
    s = ''
#    print(input_str.encode('utf-8'))
    for c in input_str:
        if c in s1:
            s += s0[s1.index(c)]
        else:
            s += c
    return s

In [232]:
remove_accents(test)

b'\xc4\x90\xe1\xbb\x97 Th\xe1\xbb\x8b B\xc3\xb4ng (VIE)  2:08.64 Q   3 '


'Do Thi Bong (VIE)  2:08.64 Q   3 '

In [ ]:
# Test

In [504]:
test2='  Rank  Team  Time  Notes   1 \xa0Japan\xa0(JPN)Ryota YamagataShuhei TadaYoshihide KiryuAsuka Cambridge 38.20 Q   2 \xa0China\xa0(CHN)Xu HaiyangMi HongSu BingtianXu Zhouzheng 38.88 Q   3 \xa0South Korea\xa0(KOR)Kim Kuk-youngMo Il-hwanOh Kyong-sooKim Min-kyun 39.34 Q   4 \xa0Hong Kong\xa0(HKG)Chan Ming TaiNg Ka FungLai Chun HoTsui Chi Ho 39.54 q   5 \xa0Philippines\xa0(PHI)Anfernee LopenaEric CrayClinton BautistaTrenten Beram 39.59    6 \xa0Oman\xa0(OMA)Fatek AdnanBarakat Al-HarthiMohamed Obaid Al-SaadiAmmar Al-Saifi 39.76    7 \xa0Kazakhstan\xa0(KAZ)Vyacheslav ZemsAlexandr KasperVitaliy ZemsVladislav Grigoryev 40.04  '

In [255]:
test3=' Rank  Team  Time  Notes   1  KazakhstanRima KashafutdinovaElina MikhinaOlga SafronovaSvetlana Golendova 44.40 Q   2  ThailandSureewan RunanOnuma ChatthaSupawan ThipatParichat Charoensuk 44.81 Q   3  JapanMidori MikaseKana IchikawaNodoka SekoMasumi Aoki 44.95 Q   4  Hong KongLam On KiLeung Kwan YiPoon Hang WaiChan Pui Kei 45.53 q   5  IndonesiaTyas MurtiningsihYulianaLusiana SatrianiFitria Indah Wahyuni 45.66    6  CambodiaChan SeyhaDuong SreypheapBen SeyhaChan Lina 50.58 '

In [505]:
pattern='\s\xa0'


delimiter=''
string=delimiter.join(test2)

res=re.split(pattern, string)

res.pop(0) # remove column names


'  Rank  Team  Time  Notes   1'

In [506]:
res

['Japan\xa0(JPN)Ryota YamagataShuhei TadaYoshihide KiryuAsuka Cambridge 38.20 Q   2',
 'China\xa0(CHN)Xu HaiyangMi HongSu BingtianXu Zhouzheng 38.88 Q   3',
 'South Korea\xa0(KOR)Kim Kuk-youngMo Il-hwanOh Kyong-sooKim Min-kyun 39.34 Q   4',
 'Hong Kong\xa0(HKG)Chan Ming TaiNg Ka FungLai Chun HoTsui Chi Ho 39.54 q   5',
 'Philippines\xa0(PHI)Anfernee LopenaEric CrayClinton BautistaTrenten Beram 39.59    6',
 'Oman\xa0(OMA)Fatek AdnanBarakat Al-HarthiMohamed Obaid Al-SaadiAmmar Al-Saifi 39.76    7',
 'Kazakhstan\xa0(KAZ)Vyacheslav ZemsAlexandr KasperVitaliy ZemsVladislav Grigoryev 40.04  ']

In [507]:
res[1]

'China\xa0(CHN)Xu HaiyangMi HongSu BingtianXu Zhouzheng 38.88 Q   3'

In [508]:
pos = re.search('[a-z][A-Z]', test2)


In [509]:
pos.start()

55

In [510]:
pos=pos.start()+1

In [511]:
name1=test2[:pos]

In [512]:
name1

'  Rank  Team  Time  Notes   1 \xa0Japan\xa0(JPN)Ryota Yamagata'

In [460]:
new_string=test2[pos:]

In [461]:
new_string

'Shuhei TadaYoshihide KiryuAsuka Cambridge 38.20'

In [462]:
pos = re.search('[a-z][A-Z]', new_string)


In [463]:
pos=pos.start()+1

In [464]:
name2=new_string[:pos]

In [465]:
name2

'Shuhei Tada'

In [263]:
# test break in relay country abbrev pattern

In [267]:
lpos2 = re.search('Notes\s\s\s\d', test3)

In [269]:
lpos2.end()

28

In [270]:
pos=lpos2.end()+1

In [271]:
names=test3[pos:]

In [272]:
names

' KazakhstanRima KashafutdinovaElina MikhinaOlga SafronovaSvetlana Golendova 44.40 Q   2  ThailandSureewan RunanOnuma ChatthaSupawan ThipatParichat Charoensuk 44.81 Q   3  JapanMidori MikaseKana IchikawaNodoka SekoMasumi Aoki 44.95 Q   4  Hong KongLam On KiLeung Kwan YiPoon Hang WaiChan Pui Kei 45.53 q   5  IndonesiaTyas MurtiningsihYulianaLusiana SatrianiFitria Indah Wahyuni 45.66    6  CambodiaChan SeyhaDuong SreypheapBen SeyhaChan Lina 50.58 '

In [211]:
# Parse relay results and put into a list

import awoc


def results_relay(data, round, wind1, heat1, gender, event, year, competition):
    
    my_world = awoc.AWOC()

    asia_countries=my_world.get_countries_list_of('Asia')
    
    global final

    temp_names=["Rank", "Name", "Country", "Result", "Qualification", "Round", "Wind", "Heat", "Gender", "Event", "Year", "Competition"]
    
    df_temp = pd.DataFrame(columns=temp_names)

# turn results into a list and strip out column headers
# replace '\n' with ' ' beforehand

    pattern='\s\xa0'

    delimiter=''
    string=delimiter.join(data)

    res=re.split(pattern, string)

    res.pop(0) # remove column names
        
# Loop for relay events

    for count, ele in enumerate(res):
    
        unicode = remove_accents(ele)
                
        lpos = re.search('\)[A-Z]', unicode) # strip accented characters. Identify three letter abbrev
        rpos = re.search('\s\d', ele)
#        npos = re.search('Notes\s\s\s\d', unicode) # identify strings without country abbreviation
    
        pattern = '\xa0'
        
        if 'DNS' in ele or 'DSQ' in ele or 'DNF' in ele or 'DQ' in ele: # skip DNS/DSQ/DNF/DQ
            
            continue
            
  ########################          
            
        if lpos:  # if three character country code exists
                            
            new_pos=lpos.start()+1 # take next character over from ) to start split
                
    # Splice string to extract name of competitor
    # Use a different process for relay events due to names being concatenated with spaces
                      
            l = ele[:new_pos] # left string post splicing
            r = ele[rpos.start():] # right string post slicing
                 
            names=ele[new_pos:rpos.start()] # extract concatenated list of competitors
    
# split the concatenated names into name_list
        
            matches = len(re.findall('[a-z][A-Z]', names)) # counts number of matches to get number of splits
    
            name_list=[]
                  
                    
            for i in range(matches): # splits concatenated string into individual names and puts into list

                name_pos = re.search('[a-z][A-Z]', names)
    
                split_pos=name_pos.start()+1
    
                name=names[:split_pos] # extract a name
        
                names=names[split_pos:] # find out what's left
        
                name_list.append(name)

            name_list.append(names) # append final name that is left     
        
#    names=re.split(name_pattern, names) # split the concatenated names
    
            country=re.split(pattern, l)[1] # split and assign country code
          
#    list=r.split(' ')
    
#    print(list)
#            
            count=str(count+1) # add one for the rank
    
            r=r.strip()  # remove whitespaces
            result=re.split(' ', r)
        
#        print(result)
    
            if len(result)>2:
            
    
                K=len(result)-2
    
                result=result[: len(result)-K] # delete last K entries
        
            elif len(result)<2:
            
                result.append(' ')  # If no Q/q flag then must be result + blank space
            
        
    
            combined=count+','+ country +','+','.join(result) # create string and use ',' as delimiter
        
            line=combined.split(",") # use split to create list from string
    
        
            line.insert(1, name_list) # insert nested name list into position 1    
        
            line.append(round)  
            line.append(wind1)  
            line.append(heat1)
            line.append(gender)
            line.append(event)
            line.append(event_year)
            line.append(competition)
                
            df_temp.loc[len(df_temp)] = line # add new row to dataframe
        
            final=df_temp
        
        else: # invoke pattern without country abbreviation. Column headers already removed.
                                                
                
    # Splice string to extract name of competitor
    # Use a different process for relay events due to names being concatenated with spaces
        
              
#            l = ele[:new_pos] # left string post splicing
            r = ele[rpos.start():] # slice out the result and qualification flag
                 
            names=ele[:rpos.start()] # extract concatenated list of countries+competitors. Country is first position

            
# split the concatenated names into name_list. Country is first entry in list
        
            matches = len(re.findall('[a-z][A-Z]', names)) # counts number of matches to get number of splits
    
            name_list=[]
                  
                    
            for i in range(matches): # splits concatenated string into individual names and puts into list

                name_pos = re.search('[a-z][A-Z]', names)
    
                split_pos=name_pos.start()+1
    
                name=names[:split_pos] # extract a name
        
                names=names[split_pos:] # find out what's left
        
                name_list.append(name)

            name_list.append(names) # append final name that is left   
            
            print(name_list)
            
        
#    Assign country and remove from list of names
    
            country=name_list[0] # country is first in split list
                    
            name_list = name_list[1:] # remove country from list of names   
          
#    list=r.split(' ')
    
#    print(list)
#            
            count=str(count+1) # add one for the rank
    
            r=r.strip()  # remove whitespaces
            result=re.split(' ', r)
        
#        print(result)
    
            if len(result)>2:
            
    
                K=len(result)-2
    
                result=result[: len(result)-K] # delete last K entries
        
            elif len(result)<2:
            
                result.append(' ')  # If no Q/q flag then must be result + blank space
            
        
    
            combined=count+','+ country +','+','.join(result) # create string and use ',' as delimiter
        
            line=combined.split(",") # use split to create list from string
    
        
            line.insert(1, name_list) # insert relay list of names into position 1    
        
            line.append(round)  
            line.append(wind1)  
            line.append(heat1)
            line.append(gender)
            line.append(event)
            line.append(event_year)
            line.append(competition)
                
            df_temp.loc[len(df_temp)] = line # add new row to dataframe
        
            final=df_temp

    
            
    
    return final

    

In [252]:
matches = re.findall('[a-z][A-Z]', names)

print(len(matches))    


NameError: name 'names' is not defined

In [239]:
import awoc

my_world = awoc.AWOC()


my_world.get_countries_list_of('Asia')

['Afghanistan',
 'Armenia',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Bhutan',
 'British Indian Ocean Territory',
 'Brunei',
 'Cambodia',
 'China',
 'Christmas Island',
 'Cocos Islands',
 'Georgia',
 'Hong Kong',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Israel',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kuwait',
 'Kyrgyzstan',
 'Laos',
 'Lebanon',
 'Macau',
 'Malaysia',
 'Maldives',
 'Mongolia',
 'Myanmar',
 'Nepal',
 'North Korea',
 'Oman',
 'Pakistan',
 'Palestine',
 'Philippines',
 'Qatar',
 'Saudi Arabia',
 'Singapore',
 'South Korea',
 'Sri Lanka',
 'Syria',
 'Taiwan',
 'Tajikistan',
 'Thailand',
 'Turkey',
 'Turkmenistan',
 'United Arab Emirates',
 'Uzbekistan',
 'Vietnam',
 'Yemen']